# Three steps

1. Save CT images
2. Save masks
3. Rename masks
4. QC

In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from pydicom import dcmread
from dicom_csv.rtstruct import get_contour_seq_name
from dicom_csv.utils import Series
from subprocess import check_call
from typing import Union


In [2]:
# !pip install dcm2niix

In [3]:
def dicom_to_nifti(source_folder: Path, target_folder: Path, name=None):
    """Converts folder of dicoms into a nii.gz file."""
    cmd = ['dcm2niix', '-z', 'y']
    if name is not None:
        cmd.append('-f')
        cmd.append(name)
    cmd.append('-o')
    cmd.append(f'{str(target_folder)}/')
    cmd.append(f'{str(source_folder)}/')
    return check_call(cmd)

In [4]:
df = pd.read_csv('/mount/attic/data/burdenko-tcia-test/meta.csv')

/tmp/ipykernel_3358037/1396226927.py:1: DtypeWarning: Columns (11,12,13,14,42,45,48,58,70,80,92,94,95,98,103,104,105,111,116,134,140,142,143,150,152,153,155,161,168,169,170,173,175,177,178,183,184,185,189,191,192,198,200,202,204,206,210,211,214,215,227,228,229,230,231,232,234,235,238,240,241,242,243,244,245,248,249,250,251,252,257,258,259,260,261,262,265,266,267,271,272,273,274,275,276,279,280,282,283,284,285,286,287,289,290,291,292,293,295,296,297,299,300,301,302,307,308,309,310,315,317,318,320,324) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/mount/attic/data/burdenko-tcia-test/meta.csv')


In [5]:
def get_series_instance_uid(rtstruct_path: [str, Path]):
    """Get SeriesInstanceUID of a reference image of input RTSTRUCT."""
    dicom = dcmread(rtstruct_path)
    return dicom.ReferencedFrameOfReferenceSequence[0]\
                .RTReferencedStudySequence[0]\
                .RTReferencedSeriesSequence[0].SeriesInstanceUID

In [6]:
source_root = Path('/mount/attic/data/burdenko-tcia-test/')
target_root = Path('/mount/attic/data/burdenko-tcia-CT')

# Save CT images as nii.gz

In [22]:
df_computed_tomography = df.query('Modality=="CT"')
df_computed_tomography = df_computed_tomography[df_computed_tomography.PathToFolder.apply(lambda x: 'Radiotherapy' in x)]

In [32]:
errors = []
for subfolder in tqdm(df_computed_tomography.drop_duplicates('SeriesInstanceUID').PathToFolder.unique()):
    patient_id = subfolder.split('/')[0]
    if not (target_root / patient_id).exists():
        (target_root / patient_id).mkdir()
    try:
        dicom_to_nifti(source_root / subfolder, target_root / patient_id, name='CT')
    except Exception as e:
        errors.append((patient_id, str(e)))

  0%|                              | 0/180 [00:00<?, ?it/s]

Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 219 DICOM file(s)
Convert 219 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/CT (512x512x219x1)


  1%|                      | 1/180 [00:02<07:04,  2.37s/it]

Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/CT.nii"
Conversion required 2.308703 seconds (0.295867 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 88 DICOM file(s)
Convert 88 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-112/CT (512x512x88x1)


  1%|▏                     | 2/180 [00:06<09:55,  3.34s/it]

...

Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-096/CT.nii"
Conversion required 4.110368 seconds (0.360138 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 96 DICOM file(s)
Convert 96 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/CT (512x512x96x1)


 99%|███████████████████▉| 179/180 [18:40<00:04,  4.77s/it]

Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/CT.nii"
Conversion required 4.299405 seconds (0.356834 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 220 DICOM file(s)
Convert 220 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-135/CT (512x512x220x1)


100%|████████████████████| 180/180 [18:49<00:00,  6.28s/it]

Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-135/CT.nii"
Conversion required 9.542616 seconds (0.772372 for core code).


In [33]:
errors

[]

# Collect contours

In [7]:
table = []
errors = []
for _, row in tqdm(df.query('Modality=="RTSTRUCT"').iterrows()):
    try:
        dcm = dcmread(source_root / row.PathToFolder / row.FileName)
        reference_uid = get_series_instance_uid(source_root / row.PathToFolder / row.FileName)
        for contour in get_contour_seq_name(dcm):
            table.append([row.PatientID, row.SeriesInstanceUID, reference_uid, contour])
    except AttributeError as e:
        errors.append(str(e))

432it [01:46,  4.06it/s]


In [8]:
df_contours = pd.DataFrame(table, columns=['PatientID', 'SeriesInstanceUID', 'ReferenceSeriesInstanceUID', 'ContourName'])

In [9]:
subset = df[df.SeriesInstanceUID.isin(df_contours.ReferenceSeriesInstanceUID.unique())].drop_duplicates(subset=['SeriesInstanceUID'])
subset = df_contours[df_contours.ReferenceSeriesInstanceUID.isin(subset.query('Modality=="CT"').SeriesInstanceUID)]

In [10]:
valid_contour_names = [
    'BODY',
    'Brain',
    'BrainStem',
    'Chiasm',
    'Eye_L',
    'Eye_R',
    'Lens_L',
    'Lens_R',
    'OpticNerve_L',
    'OpticNerve_R',
    'body volume',
    'brain stem',
    'chiasma',
    'left eye',
    'left optic nerve',
    'left pupil',
    'right eye',
    'right optic nerve',
    'right pupil',
]

In [12]:
subset = subset[subset.ContourName.isin(valid_contour_names)]

In [13]:
subset['ContourNameFixed'] = subset['ContourName'].map(
    {'BODY':'BODY',
    'Brain':'Brain',
    'BrainStem':'BrainStem',
    'Chiasm':'Chiasm',
    'Eye_L':'Eye_L',
    'Eye_R':'Eye_R',
    'Lens_L':'Lens_L',
    'Lens_R':'Lens_R',
    'OpticNerve_L':'OpticNerve_L',
    'OpticNerve_R':'OpticNerve_R',
    'body volume':'BODY',
    'brain stem':'BrainStem',
    'chiasma':'Chiasm',
    'left eye':'Eye_L',
    'left optic nerve':'OpticNerve_L',
    'left pupil':'Lens_L',
    'right eye':'Eye_R',
    'right optic nerve':'OpticNerve_R',
    'right pupil':'Lens_R',}
)

In [14]:
subset['ContourNameFixed'].value_counts()

BODY            193
BrainStem       193
Chiasm          193
Eye_L           193
Eye_R           193
Lens_L          193
Lens_R          193
OpticNerve_L    193
OpticNerve_R    193
Brain            96
Name: ContourNameFixed, dtype: int64

# Contour to image

In [15]:
from dicom_csv import  order_series, stack_images
from dicom_csv.rtstruct import contours_to_image
import numpy as np

In [16]:
def read_image_series(uid):
    sub_uid = df_computed_tomography[df_computed_tomography.SeriesInstanceUID == uid]
    ct_series = [dcmread(source_root / ds.PathToFolder / ds.FileName)\
             for _,ds in sub_uid.iterrows()]
    return order_series(ct_series)

In [17]:
def get_contours(uid):
    uid_rtstruct = subset[subset.ReferenceSeriesInstanceUID==uid].SeriesInstanceUID.values[0]
    row = df.query('SeriesInstanceUID==@uid_rtstruct')

    ct_series = read_image_series(uid)
    rtstruct = dcmread(source_root / row.PathToFolder.values[0] / row.FileName.values[0])
    contours = contours_to_image(ct_series, rtstruct)
    
    return ct_series, contours

In [18]:
def mask_to_dicom(series, mask, folder):
    """Writes three-dimensional binary mask to series of reference DICOM files and save them to folder."""
    dtype = series[0].pixel_array.dtype
    for frame, m2 in zip(series, np.moveaxis(mask, 2, 0)):
        byte_mask = m2.astype(dtype).tobytes()
        uid = frame.SOPInstanceUID
        if len(byte_mask) % 2 != 0:
            byte_mask += b'\x00'
        frame.PixelData = byte_mask
        frame.save_as(folder / f'{uid}.dcm')

In [19]:
mask_names_dict = {'BODY':'BODY',
    'Brain':'Brain',
    'BrainStem':'BrainStem',
    'Chiasm':'Chiasm',
    'Eye_L':'Eye_L',
    'Eye_R':'Eye_R',
    'Lens_L':'Lens_L',
    'Lens_R':'Lens_R',
    'OpticNerve_L':'OpticNerve_L',
    'OpticNerve_R':'OpticNerve_R',
    'body volume':'BODY',
    'brain stem':'BrainStem',
    'chiasma':'Chiasm',
    'left eye':'Eye_L',
    'left optic nerve':'OpticNerve_L',
    'left pupil':'Lens_L',
    'right eye':'Eye_R',
    'right optic nerve':'OpticNerve_R',
    'right pupil':'Lens_R',}

In [168]:
errors = []
for uid in tqdm(subset.ReferenceSeriesInstanceUID.unique()):
    try:
        patient_id = subset[subset.ReferenceSeriesInstanceUID==uid].PatientID.values[0]
        ct_series, contours = get_contours(uid)
        
        for key in tqdm(contours.keys()):
            if key[0] in valid_contour_names:
                mask = contours[key].get_mask()
                folder = target_root / patient_id / mask_names_dict[key[0]]
                folder.mkdir()
                mask_to_dicom(ct_series, mask, folder)
        
    except Exception as e:
        errors.append((uid, str(e)))

 58%|█████████████▍         | 7/12 [00:07<00:02,  1.87it/s]


 53%|████████████▎          | 8/15 [00:38<00:13,  1.99s/it]


 50%|███████████▌           | 6/12 [00:05<00:03,  1.53it/s]


 85%|██████████████████▌   | 11/13 [00:54<00:02,  1.48s/it]


 64%|██████████████▋        | 7/11 [00:08<00:02,  1.70it/s]


 42%|█████████▌             | 5/12 [00:06<00:05,  1.32it/s]


 18%|████▏                  | 2/11 [00:08<00:32,  3.66s/it]


100%|██████████████████| 179/179 [2:04:03<00:00, 41.58s/it]


In [171]:
errors

[('1.3.6.1.4.1.14519.5.2.1.60188900259336818950932039755931426115',
  "'Dataset' object has no attribute 'ROIName'"),
 ('1.3.6.1.4.1.14519.5.2.1.124198430859518220773959056954678901886',
  "[Errno 17] File exists: '/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-141/BODY'"),
 ('1.3.6.1.4.1.14519.5.2.1.160947619329483928301402181961684012929',
  "[Errno 17] File exists: '/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-143/BODY'"),
 ('1.3.6.1.4.1.14519.5.2.1.188230566712732260167691526005736634425',
  "[Errno 17] File exists: '/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-010/BODY'"),
 ('1.3.6.1.4.1.14519.5.2.1.289835101549031962468573828840915449048',
  "'Dataset' object has no attribute 'ROIName'"),
 ('1.3.6.1.4.1.14519.5.2.1.105298092945252540981863706010491048129',
  "[Errno 17] File exists: '/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-178/BODY'"),
 ('1.3.6.1.4.1.14519.5.2.1.111977347583659661715572287945998324598',
  "[Errno 17] File exists: '/mount/attic/data/burdenko-tcia-CT

# Convert dicom masks to nii.gz

In [26]:
errors2 = []
for folder in tqdm(list(target_root.glob('*'))[104:]):
    for subfolder in mask_names_dict.values():
        if (folder / subfolder).exists():
            try:
                dicom_to_nifti(folder / subfolder, folder, name=subfolder)
            except Exception as e:
                errors2.append((subfolder, str(e)))

  0%|                               | 0/76 [00:00<?, ?it/s]

Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 213 DICOM file(s)
Convert 213 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-140/BODYb (512x512x213x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-140/BODYb.nii"
Conversion required 7.059019 seconds (0.431245 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 213 DICOM file(s)
Convert 213 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-140/BrainStemb (512x512x213x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-140/BrainStemb.nii"
Conversion required 7.500420 seconds (0.423330 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 213 DICOM file(s)
Convert 213 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-140/Chiasmb (512x512x213x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/dat

  1%|▎                   | 1/76 [02:07<2:38:55, 127.14s/it]

Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-140/Lens_Rc.nii"
Conversion required 7.396011 seconds (0.444302 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 80 DICOM file(s)
Convert 80 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-060/BODYb (512x512x80x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-060/BODYb.nii"
Conversion required 2.722962 seconds (0.160625 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 80 DICOM file(s)
Convert 80 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-060/BrainStemb (512x512x80x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-060/BrainStemb.nii"
Conversion required 2.685683 seconds (0.162635 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 80 DICOM 

  3%|▌                    | 2/76 [02:43<1:30:49, 73.64s/it]

...

Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-096/Lens_Ra.nii"
Conversion required 0.241426 seconds (0.138548 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 96 DICOM file(s)
Convert 96 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/BODY (512x512x96x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/BODY.nii"
Conversion required 3.139274 seconds (0.234097 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 96 DICOM file(s)
Convert 96 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/BrainStem (512x512x96x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/BrainStem.nii"
Conversion required 3.156090 seconds (0.212642 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 96 DICOM file

 99%|███████████████████▋| 75/76 [1:18:21<00:40, 40.27s/it]

Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 96 DICOM file(s)
Convert 96 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/Lens_Ra (512x512x96x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-092/Lens_Ra.nii"
Conversion required 0.233861 seconds (0.136381 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 220 DICOM file(s)
Convert 220 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-135/BODY (512x512x220x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-135/BODY.nii"
Conversion required 7.460685 seconds (0.692086 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 220 DICOM file(s)
Convert 220 DICOM as /mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-135/Brain (512x512x220x1)
Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tc

100%|████████████████████| 76/76 [1:19:46<00:00, 62.98s/it]

Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-135/Lens_Ra.nii"
Conversion required 0.521600 seconds (0.312931 for core code).


In [27]:
errors2

[]

In [169]:
# dicom_to_nifti('/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/Eye_L', '/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/', name='Eye_L')
# dicom_to_nifti('/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/Eye_R', '/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/', name='Eye_R')
# dicom_to_nifti('/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/Brain', '/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/', name='Brain')

In [164]:
# from nibabel import load

In [165]:
# eye_l = load('/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/Eye_L.nii.gz').get_fdata()
# eye_r = load('/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/Eye_R.nii.gz').get_fdata()
# brain = load('/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/Brain.nii.gz').get_fdata()

# im = load('/mount/attic/data/burdenko-tcia-CT/Burdenko-GBM-174/CT.nii.gz').get_fdata()

In [166]:
# from dpipe.im.visualize import slice3d

In [170]:
# slice3d(im, brain, eye_l, eye_r)

# Copy to separate folder

In [28]:
mask_names_dict.values()

dict_values(['BODY', 'Brain', 'BrainStem', 'Chiasm', 'Eye_L', 'Eye_R', 'Lens_L', 'Lens_R', 'OpticNerve_L', 'OpticNerve_R', 'BODY', 'BrainStem', 'Chiasm', 'Eye_L', 'OpticNerve_L', 'Lens_L', 'Eye_R', 'OpticNerve_R', 'Lens_R'])

In [29]:
source_folder = Path('/mount/attic/data/burdenko-tcia-CT')
target_folder = Path('/mount/attic/data/burdenko-tcia-CT-niigz')

In [30]:
import shutil

In [37]:
masks_to_copy = [
    'BrainStem',
'Chiasm',
'Eye_L',
'Eye_R',
'Lens_L',
'Lens_R',
'OpticNerve_L', 
'OpticNerve_R'
]

In [38]:
errors = []
for folder in tqdm(source_folder.glob('*')):
    try:
        target_copy = target_folder / folder.name
        (target_copy).mkdir()
        shutil.copyfile(folder / 'CT.nii.gz', target_copy / 'CT.nii.gz')
        for mask in masks_to_copy:
            try:
                shutil.copyfile(folder / f'{mask}.nii.gz', target_copy / f'{mask}.nii.gz')
            except:
                pass        
    except Exception as e:
        errors.append((folder.name, str(e)))

180it [01:07,  2.67it/s]


# Convert to single mask

In [6]:
# import nibabel as nib
# from pathlib import Path
# import numpy as np
# from tqdm import tqdm

# target_folder = Path('/mount/attic/data/burdenko-tcia-CT-niigz')

# def fix_image_shift(array):
#     min_value = np.unique(array).min()
#     return array - min_value

# masks_to_copy = {
#     'BrainStem':1,
#     'Chiasm':2,
#     'Eye_L':3,
#     'Eye_R':4,
#     'Lens_L':5,
#     'Lens_R':6,
#     'OpticNerve_L':7, 
#     'OpticNerve_R':8
# }

# # 027, 112, 117, 130, some errors with masks

# errors = []
# for folder in tqdm(target_folder.glob('*')):
#     try:
#         shape = nib.load(folder / 'CT.nii.gz').shape
#         masks_array = np.zeros(shape)
#         for mask, i in masks_to_copy.items():
#             m_nii = nib.load(folder / f'{mask}.nii.gz')
#             m_array = m_nii.get_fdata()
#             m_array_fixed = fix_image_shift(m_array) * i
#             masks_array = np.maximum(masks_array, m_array_fixed)
#         masks_array_nii = nib.Nifti1Image(masks_array.astype(np.int8), m_nii.affine, m_nii.header)
#         nib.save(masks_array_nii, folder / 'masks.nii.gz')
#     except Exception as e:
#         errors.append((folder.name, str(e)))

# rm Redundant files

In [4]:
import os
from pathlib import Path
from tqdm import tqdm

target_folder = Path('/mount/attic/data/burdenko-tcia-CT-niigz')

In [6]:
masks_to_copy = {
    'BrainStem':1,
    'Chiasm':2,
    'Eye_L':3,
    'Eye_R':4,
    'Lens_L':5,
    'Lens_R':6,
    'OpticNerve_L':7, 
    'OpticNerve_R':8
}

In [7]:
# for subfolder in tqdm(target_folder.glob('*')):
#     for mask in masks_to_copy.keys():
#         os.remove(subfolder / f'{mask}.nii.gz')

176it [00:00, 756.49it/s]
